# Setup

In [5]:
import torch
from models import DummyGPTModel, GPTConfig
import tiktoken

In [ ]:
GPTConfig(
    vocab_size=50257,
    context_length=1024,
    embed_dim=768,
    n_heads=12,
    n_layers=12,
    drop_rate=0.1,
    qkv_bias=False
)


# Dummy implementation

## Setting up tokens

In [6]:
tokenizer = tiktoken.get_encoding("gpt2")
batch: list[torch.Tensor] = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

batch = torch.stack(batch, dim=0)  # converts list of tensors to a single tensor with dimension 2*4
batch

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

## Initializing a 124-million param gpt mdel

In [ ]:
torch.manual_seed(123)
model = DummyGPTModel()